In [1]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [2]:
# save_path = '/content/drive/MyDrive/Dl projects/RNN basics/Machine Translation'

model_name1 = 'rnn_encoder_model'
# model_path1 = save_path + '/' + model_name1
encoder_model = tf.keras.models.load_model(model_name1)

model_name2 = 'rnn_decoder_model'
# model_path2 = save_path + '/' + model_name2
decoder_model = tf.keras.models.load_model(model_name2)

In [3]:
# import os
# os.chdir(save_path)
# print(os.getcwd())

import joblib
tags_dict = joblib.load('tags.joblib')
word_tokenizer = joblib.load('tokenizer.joblib')

print('\n')
print(tags_dict)



{1: '<OOV>', 2: ' ', 3: 'e', 4: 'i', 5: 'n', 6: 't', 7: 's', 8: 'h', 9: 'a', 10: 'r', 11: 'c', 12: 'm', 13: 'd', 14: 'u', 15: 'o', 16: 'l', 17: 'g', 18: 'b', 19: 'w', 20: 'f', 21: 'k', 22: 'z', 23: 'ü', 24: 'v', 25: 'p', 26: 'ä', 27: 'ö', 28: 'ß', 29: 'j', 30: 'y', 31: '’', 32: 'x', 33: '0', 34: '3', 35: '1', 36: 'q', 37: '2', 38: '„', 39: '“', 40: '8', 41: '9', 42: '\xa0', 43: '5', 44: '6', 45: '7', 46: '4', 47: '\u200b', 48: 'ō', 49: 'á', 50: '—', 51: '–', 52: '\u202f', 0: 'padding'}


In [4]:
import string
import contractions
import numpy as np

def text_preprocess(s):
    s = contractions.fix(s)
    s = s.lower()
    s = s.rstrip()
    s = s.lstrip()

    text = ""
    for ch in s:
        if ch not in string.punctuation:
            text = text + ch
    return text

In [5]:
def translate_sentence(text, max_len, start_token, end_token, max_output_len, tags, word_tokenizer):
    original_text = text
    text = text_preprocess(text)

    data = np.array(word_tokenizer.texts_to_sequences(text))
    data = data.ravel()

    data = tf.keras.preprocessing.sequence.pad_sequences(
        [data], maxlen = max_len, padding = 'pre',
        truncating='post', value=0.0 )
  
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = start_token
    target_seq = target_seq.ravel()

    encoder_outputs, h, c = encoder_model.predict(data)
    states_value = [h,c]
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value + [encoder_outputs])

    output_sen = []
    for _ in range(max_output_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value + [encoder_outputs])
        idx = np.argmax(output_tokens,axis=2).ravel()

        if idx == end_token:
            break
        else:
            output_sen.append(tags[idx[0]])

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = idx[0]
        target_seq = target_seq.ravel()
        states_value = [h, c]

    return original_text,''.join(output_sen)

In [6]:
text = 'I am very Happy'
max_len = 21
start_token = 53
end_token = 54
max_output_len = 120

In [7]:
text, output = translate_sentence(text, max_len, start_token, end_token, max_output_len, tags_dict, word_tokenizer) 

In [8]:
print(text)
print(output)

I am very Happy
ich bin sehr glücklich


In [9]:
input_list = []
input_list.append('Input text in English')
input_list.append(text)

output_list = []
output_list.append('Translated text in German')
output_list.append(output)

In [10]:
print(input_list)
print(output_list)

['Input text in English', 'I am very Happy']
['Translated text in German', 'ich bin sehr glücklich']


In [11]:
from pywebio.input import input,TEXT
from pywebio.output import put_tabs,put_table

In [12]:
def predict_output():
    text = input('Machine Language Translation English to German',placeholder = "Enter the text in English", type = TEXT)
    
    max_len = 21
    start_token = 53
    end_token = 54
    max_output_len = 120
    text, output = translate_sentence(text, max_len, start_token, end_token, max_output_len, tags_dict, word_tokenizer) 
    
    input_list = []
    input_list.append('Input text in English')
    input_list.append(text)

    output_list = []
    output_list.append('Translated text in German')
    output_list.append(output)
    
    final_list = []
    final_list.append(input_list)
    final_list.append(output_list)
    
    put_table(final_list, header=['Description', 'Text'])

In [13]:
predict_output()